In [1]:
try:
    import sys
    import os
    sys.path.insert(0, os.path.abspath(os.curdir))
except ModuleNotFoundError:
    pass
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pyspark.sql.functions as f
import pyspark.sql.types as t
import os
from functools import reduce
from datetime import datetime, time
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import lag
from pyspark.sql.window import Window
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import DoubleType
import pandas as pd
import math
import folium
import carregar_mapa_desagrupados as cmd
import carregar_dados_agrupados as cda


<p>1 - Iniciar sessão spark</p>

In [2]:
spark_session = cda.iniciar_sessao_spark()

23/09/30 18:27:12 WARN Utils: Your hostname, DESKTOP-N81ME1R resolves to a loopback address: 127.0.1.1; using 172.24.108.240 instead (on interface eth0)
23/09/30 18:27:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/30 18:27:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
dataframe_desagrupados = cmd.load_dataframe_desagrupados(spark=spark_session)
dataframe_desagrupados.show()

+-----------+--------------------+--------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+-----------------+-----+
|CODIGO_AREA|            CONSOCIO|             EMPRESA|DATA_EXTRACAO|HORA_API|LETREIRO_COMPLETO|SENTIDO_OPERACAO|CODIGO_IDENTIFICADOR_LINHA|PREFIXO_ONIBUS|DATA_HORA_CAPTURA_LOCALIZACAO|           LATITUDE|          LONGITUDE|DATA_EXTRACAO_API|TURNO|
+-----------+--------------------+--------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+-----------------+-----+
|          4|PÊSSEGO TRANSPORT...|PÊSSEGO TRANSPORT...|   2023-09-15|   08:50|          3054-10|               2|                     33743|         47399|         2023-09-15T11:50:12Z|         -23.598638|         -46.416157|       2023-09-15|Manhã|


In [4]:
df_prefixo_onibus = cmd.obter_prefixo_onibus(dataframes_desagrupados_completo=dataframe_desagrupados)
df_prefixo_onibus

[66580,
 15173,
 73951,
 78598,
 26486,
 61679,
 36102,
 31944,
 66070,
 11567,
 48280,
 22129,
 31703,
 15057,
 26543,
 77605,
 36103,
 45298,
 55474,
 66114,
 63212,
 31156,
 64442,
 71655,
 32098,
 68463,
 56242,
 52743,
 72935,
 52001,
 10422,
 61749,
 71530,
 48899,
 51418,
 21965,
 63676,
 52910,
 51428,
 64317,
 31762,
 11745,
 51056,
 72485,
 15194,
 45166,
 72926,
 45410,
 25207,
 16530,
 55426,
 45726,
 71429,
 16597,
 85760,
 52051,
 71285,
 71292,
 47745,
 71386,
 81572,
 36077,
 47050,
 71549,
 16673,
 81443,
 48963,
 52904,
 76085,
 77441,
 71548,
 45390,
 26140,
 85641,
 15221,
 68166,
 68047,
 63181,
 63448,
 86153,
 66911,
 73916,
 71255,
 51105,
 68752,
 76050,
 11771,
 47741,
 48203,
 63678,
 45724,
 63586,
 81038,
 55329,
 55093,
 77434,
 11860,
 45558,
 68314,
 51112,
 72928,
 31159,
 32204,
 47683,
 15133,
 11988,
 31111,
 47636,
 63109,
 47229,
 78203,
 52745,
 22808,
 31126,
 68608,
 45134,
 31102,
 80978,
 31832,
 36577,
 45473,
 16499,
 22707,
 45580,
 45474,


In [5]:
prefixo_onibus = '11567'

In [6]:
dataframe_filter = cmd.dataframe_filter_desagrupados(
    dataframes_desagrupados_completo=dataframe_desagrupados,
    data_extracao='2023-09-18',
    prefixo_onibus=prefixo_onibus,
    turno='Noite'
)
dataframe_filter.show()

+--------+--------------+--------------------------+-----------------+-------------------+-------------------+
|HORA_API|PREFIXO_ONIBUS|CODIGO_IDENTIFICADOR_LINHA|LETREIRO_COMPLETO|           LATITUDE|          LONGITUDE|
+--------+--------------+--------------------------+-----------------+-------------------+-------------------+
|   18:00|         11567|                     33357|          975A-10|         -23.572305|-46.642962499999996|
|   18:05|         11567|                     33357|          975A-10|-23.567685312499997|-46.648865812500006|
|   18:10|         11567|                     33357|          975A-10|       -23.56325125|-46.654227000000006|
|   18:15|         11567|                     33357|          975A-10|-23.559906624999996|        -46.6581005|
|   18:20|         11567|                     33357|          975A-10|-23.559697874999998| -46.65833550000001|
|   18:32|         11567|                     33357|          975A-10|        -23.5407145|-46.670465500000006|
|

In [7]:
letreiro_onibus = cmd.obter_letreiro_onibus(dataframe_prefixo_onibus=dataframe_filter)
letreiro_onibus

'975A-10'

In [8]:
rota_linha = cmd.obter_rota_linha(letreiro_onibus=letreiro_onibus)
rota_linha[0]

['975A-10', 'Vl. Brasilândia - Metrô Ana Rosa', '509E2F']

In [9]:
dataframe_filter = dataframe_filter.withColumn("DISTANCIA", cmd.haversine_udf(dataframe_filter["LATITUDE"], dataframe_filter["LONGITUDE"]))
dataframe_filter.show()

+--------+--------------+--------------------------+-----------------+-------------------+-------------------+-------------------+
|HORA_API|PREFIXO_ONIBUS|CODIGO_IDENTIFICADOR_LINHA|LETREIRO_COMPLETO|           LATITUDE|          LONGITUDE|          DISTANCIA|
+--------+--------------+--------------------------+-----------------+-------------------+-------------------+-------------------+
|   18:00|         11567|                     33357|          975A-10|         -23.572305|-46.642962499999996|                0.0|
|   18:05|         11567|                     33357|          975A-10|-23.567685312499997|-46.648865812500006| 0.7911142323286782|
|   18:10|         11567|                     33357|          975A-10|       -23.56325125|-46.654227000000006| 0.7359822657852066|
|   18:15|         11567|                     33357|          975A-10|-23.559906624999996|        -46.6581005|  0.542388115268007|
|   18:20|         11567|                     33357|          975A-10|-23.559697874

In [10]:
soma_distancias = dataframe_filter.selectExpr("sum(DISTANCIA)").collect()[0][0]
print(f'Soma das distâncias: {soma_distancias:.2f} km')

Soma das distâncias: 14.60 km


In [11]:
rota_linha

[['975A-10', 'Vl. Brasilândia - Metrô Ana Rosa', '509E2F']]

In [12]:
id_tracado_linha = cmd.obter_id_tracado_linha(spark_session=spark_session, rota_id=rota_linha[0][0])
id_tracado_linha

[59905, 81295]

In [13]:
tracado_linha = cmd.obter_tracado_linha(spark_session=spark_session,shape_id=id_tracado_linha)
tracado_linha

[[-23.46904754638672, -46.69623947143555],
 [-23.469057083129883, -46.69619369506836],
 [-23.469079971313477, -46.69611358642578],
 [-23.46910285949707, -46.69590377807617],
 [-23.4691104888916, -46.695743560791016],
 [-23.4691162109375, -46.69560623168945],
 [-23.469106674194336, -46.6954345703125],
 [-23.46906280517578, -46.69523620605469],
 [-23.4689998626709, -46.69501495361328],
 [-23.4689884185791, -46.69493865966797],
 [-23.4689884185791, -46.69484329223633],
 [-23.46898651123047, -46.69475555419922],
 [-23.4689884185791, -46.69470977783203],
 [-23.4689998626709, -46.69463348388672],
 [-23.469022750854492, -46.69456100463867],
 [-23.46904754638672, -46.69451141357422],
 [-23.46915626525879, -46.694427490234375],
 [-23.469280242919922, -46.6943359375],
 [-23.469337463378906, -46.69429397583008],
 [-23.469409942626953, -46.69422149658203],
 [-23.46955680847168, -46.694061279296875],
 [-23.470060348510742, -46.69347381591797],
 [-23.47023582458496, -46.69325637817383],
 [-23.470315

In [14]:
def obter_posicao_onibus_teste(df_posicao: DataFrame):
    posicao_onibus = df_posicao.rdd.flatMap(
        lambda linha: [[(linha.HORA_API), [linha.LATITUDE, linha.LONGITUDE]]]
    ).collect()

    return posicao_onibus

In [15]:
posicao_onibus = cmd.obter_posicao_onibus(df_posicao=dataframe_filter)
posicao_onibus[0]

['18:00', [-23.572305, -46.642962499999996]]

In [16]:
rota_linha

[['975A-10', 'Vl. Brasilândia - Metrô Ana Rosa', '509E2F']]

In [17]:
mapa = cmd.gerar_mapa(posicao_onibus=posicao_onibus, tracado_linha=tracado_linha,cor_linha=rota_linha[0][2], prefixo_onibus=prefixo_onibus)
mapa